In [1]:
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    return soup


In [3]:
def scrape_one_page(job_link):
    data_list = []
    for job_list in job_link:
        single_page_scrape = scrape_page(job_list)

        container = single_page_scrape.find_all('div',class_='container')

        #Job Title
        job_title = container[0].find('h1')
        job_title = job_title.text

        #Job Url
        job_url = job_list

        #Company Name
        company_name = container[0].find('p')
        company_name = company_name.text.strip()

        #Location
        location = container[0].find('span')
        location = location.text.strip()

        job_content = single_page_scrape.find('section',{'class':'content'})
        ul = job_content.find_all('ul')

        if ul and len(ul)>=3:
        #Key Responsibilites
             key_responsibilites=ul[0].text 
        
        #Reuirements
             requirements = ul[1].text
        
        #Skills and Qualification
             skills_and_qualifications = ul[2].text

        job_type = container[0].find('ul',class_='details')
        job_type = job_type.find_all('li')
        for item in job_type:
            job_type = item.text.strip()

        #Job posted date
        posted_time  = single_page_scrape.find_all('ul',class_='tags')
        posted_time = posted_time[0].find('li').text.strip()
        date_format = "%Y-%m-%d %H:%M:%S.%f"
        if "days" in  posted_time :
            job_posted_date = datetime.now() - timedelta(days=int(posted_time.split()[0]))
        elif "hours" in posted_time:
            job_posted_date = datetime.now() - timedelta(hours=int(posted_time.split()[0]))
        # job_posted_dates.append(job_posted_date.strftime(date_format))
        job_posted_dates = job_posted_date.strftime(date_format)

        
        data = {
            'job_title':job_title,
            'job_url':job_url,
            'company_name':company_name,
            'location':location,
            'job_type':job_type,
            'job_posted_on':job_posted_dates,
            'key_responsibilites':key_responsibilites,
            'requirements':requirements,
            'skills_and_qualifications':skills_and_qualifications
        }

        data_list.append(data)
    print(data_list)
    return data_list

        

In [4]:

job_title = 'Data Engineer'
l='Texas'
i=1
last_date=1
response = requests.get(f"https://www.careerjet.com/{job_title}-jobs/{l}-20747.html?nw={last_date}")
soup = BeautifulSoup(response.content,'html.parser')


In [5]:
total_jobs_txt = soup.find('p',{'class':'col col-xs-12 col-m-4 col-m-r cr'})
total_jobs_txt = total_jobs_txt.text.strip()
job_number = int(''.join(filter(str.isdigit,total_jobs_txt)))
print(job_number)

number_of_pages = math.ceil((job_number)/20)
number_of_pages


30


2

In [6]:
pn=1
job_link = []

while pn!=number_of_pages:   #for testing purpose using pn=3, to scrape up to all page condition should be pn!=number_of_pages
    scrape = scrape_page(f"https://www.careerjet.com/{job_title}-jobs/{l}-20747.html?p={pn}&nw={last_date}")
#    
    ul = scrape.find('ul',{'class':'jobs'})
    h_2 = ul.find_all('h2')
    for item in h_2:
        full_job_url = "https://www.careerjet.com"+item.a.get('href')
        job_link.append(full_job_url)
   
    # print(a)
    pn = pn+1   

print(job_link)

['https://www.careerjet.com/jobad/usae60147eacd1e4d57dfec96a108b5e27', 'https://www.careerjet.com/jobad/us1cafa27e7c6c7781d2982a4c61a182dd', 'https://www.careerjet.com/jobad/us5f4ff3af0e17b36413087be40657f1a3', 'https://www.careerjet.com/jobad/us5a57cf8d2eac117feab09d236e296469', 'https://www.careerjet.com/jobad/us887f1500fba6b52a965577fb7ed19e59', 'https://www.careerjet.com/jobad/us15091b7a1d4f6a7f5dd127f87883f28e', 'https://www.careerjet.com/jobad/us8fa5efcf67839a010aa882d2de145fb7', 'https://www.careerjet.com/jobad/us8a024752e98a9fe2b5b5e82bd4b096d2', 'https://www.careerjet.com/jobad/us6658bd41221b71b8401913bd57350c41', 'https://www.careerjet.com/jobad/us42d2cdf632cec556308451d0e93bd7e3', 'https://www.careerjet.com/jobad/use03b196a78ebd9b1e9e4e89107048fe6', 'https://www.careerjet.com/jobad/us78fcb95ddea1e68cd8b9e866956b7a63', 'https://www.careerjet.com/jobad/us058f015ac2e3e2e5b07001914fe039e8', 'https://www.careerjet.com/jobad/us1922f587d5d4e23a6c655fb19ce44ac6', 'https://www.career

In [7]:
print(len(job_link))

20


In [7]:
page_data = scrape_one_page(job_link)

[{'job_title': 'Senior Data Engineer', 'job_url': 'https://www.careerjet.com/jobad/usae60147eacd1e4d57dfec96a108b5e27', 'company_name': 'Shell', 'location': 'Houston, TX', 'job_type': 'Full-time', 'job_posted_on': '2023-07-17 09:56:10.713457', 'key_responsibilites': ' Communicate with both technical developers and business managers and gain respect and trust of leaders and staff. Actively deliver the roll-out and embedding of Data Foundation initiatives in support of the key business programs advising on the technology and using leading market standard tools. Coordinate the change management process, incident management and problem management process. Ensure traceability of requirements from Data through testing and scope changes, to training and transition. Drive implementation efficiency and effectiveness across the pilots and future projects to minimize cost, increase speed of implementation and maximize value delivery. ', 'requirements': ' Must have legal authorization to work in t

In [8]:
df = pd.DataFrame(page_data)
df


,job_title,job_url,company_name,location,job_type,job_posted_on,key_responsibilites,requirements,skills_and_qualifications
0,Senior Data Engineer,https://www.careerjet.com/jobad/usae60147eacd1...,Shell,"Houston, TX",Full-time,2023-07-17 09:56:10.713457,Communicate with both technical developers an...,Must have legal authorization to work in the ...,We’re creating a space where people with disa...
1,"Sr. Electrical Engineer - Houston, TX",https://www.careerjet.com/jobad/us1cafa27e7c6c...,Computronics Solutions,"Houston, TX",Full-time,2023-07-17 04:56:15.044041,Industry-consensus codes and standards from N...,Bachelor’s degree (B.S.) in Electrical Engine...,Recruiter Screen Manager Screen Small Panel
2,Oracle Software Development Engineer (OIC),https://www.careerjet.com/jobad/us5f4ff3af0e17...,NTT Data,"Plano, TX",Full-time,2023-07-17 11:56:16.457666,Industry-consensus codes and standards from N...,Bachelor’s degree (B.S.) in Electrical Engine...,Recruiter Screen Manager Screen Small Panel
3,Oracle Software Development Engineer (OIC),https://www.careerjet.com/jobad/us5a57cf8d2eac...,NTT Data,"Plano, TX",Full-time,2023-07-17 03:56:18.111989,Industry-consensus codes and standards from N...,Bachelor’s degree (B.S.) in Electrical Engine...,Recruiter Screen Manager Screen Small Panel
4,Azure DevOps Engineer,https://www.careerjet.com/jobad/us887f1500fba6...,AT&T,"Dallas, TX",Full-time,2023-07-17 04:56:19.687908,Industry-consensus codes and standards from N...,Bachelor’s degree (B.S.) in Electrical Engine...,Recruiter Screen Manager Screen Small Panel
5,Process / Manufacturing Engineer I,https://www.careerjet.com/jobad/us15091b7a1d4f...,Stratasys,"Belton, TX",Full-time,2023-07-17 08:56:20.942764,Assess risk and provide solutions on technica...,Due to the nature of products made at Stratas...,"Experience with PPAP manufacturing, control p..."
6,Sr. Network Engineer,https://www.careerjet.com/jobad/us8fa5efcf6783...,LyondellBasell,"Houston, TX",Full-time,2023-07-17 07:56:22.484668,Assess risk and provide solutions on technica...,Due to the nature of products made at Stratas...,"Experience with PPAP manufacturing, control p..."
7,Reliability Engineer - Mechanical,https://www.careerjet.com/jobad/us8a024752e98a...,Graphic Packaging International,"Texarkana, TX",Full-time,2023-07-17 05:56:23.962768,Analyzes maintenance and reliability work pra...,Bachelor's Degree in Mechanical or Reliabilit...,Knowledgeable in process safety programs incl...
8,Engineer Process,https://www.careerjet.com/jobad/us6658bd41221b...,Cemex,"New Braunfels, TX",Full-time,2023-07-17 04:56:25.355444,"Work following the Mission, Vision and Values...",Eliminate equipment restrictions Implement pr...,Identify Training requirements and execute Tr...
9,Operating Engineer,https://www.careerjet.com/jobad/us42d2cdf632ce...,Jones Lang LaSalle,"Fort Worth, TX",Full-time,2023-07-17 04:56:27.534548,"Work following the Mission, Vision and Values...",Eliminate equipment restrictions Implement pr...,Identify Training requirements and execute Tr...


In [ ]:
from datetime import datetime, timedelta

# Convert dynamic variable to datetime object
var = ['11 hours ago','5 days ago']
date_format = "%Y-%m-%d %H:%M:%S.%f"
job_posted_dates = []
# Convert dynamic variables to datetime objects
date_format = "%Y-%m-%d %H:%M:%S.%f"
job_posted_dates = []
for dynamic_var in var:
    if "days" in dynamic_var:
        job_posted_date = datetime.now() - timedelta(days=int(dynamic_var.split()[0]))
    elif "hours" in dynamic_var:
        job_posted_date = datetime.now() - timedelta(hours=int(dynamic_var.split()[0]))
    job_posted_dates.append(job_posted_date.strftime(date_format))

# Print job posted dates
print(job_posted_dates)
